#   MTCNN Face Detection - Implantação

*   Este componente utiliza a biblioteca [facenet-pytorch](https://github.com/timesler/facenet-pytorch), a qual disponibiliza o algorítimo [MTCNN](https://arxiv.org/abs/1604.02878). 

* O MTCNN possui a performance estado da arte nos benchmarks [FDDB](http://vis-www.cs.umass.edu/fddb/) e [WIDER FACE](http://shuoyang1213.me/WIDERFACE/)

*   Melhores explicações são encontradas neste [artigo do kaggle](https://www.kaggle.com/timesler/guide-to-mtcnn-in-facenet-pytorch)


### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
!pip install facenet-pytorch --quiet

In [ ]:
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/cv-mtcnn-face-detection/mtcnn.py

In [ ]:
%%writefile Model.py
import joblib
import nltk
import pandas as pd
import numpy as np
import base64
import cv2
from mtcnn import MTCNN_Model

        
class Model(object):
    
    def __init__(self):
        artifacts = joblib.load("/tmp/data/mtcnn.joblib")
        self.general_parameters = artifacts["general_parameters"]
        self.model_parameters = artifacts["model_parameters"]
        self.inference_parameters = artifacts["inference_parameters"]
        
    def class_names(self):
        
        return ['bboxcoord1', 'bboxcoord2', 'bboxcoord3', 'bboxcoord4', 'probability']
        
    def format_result(self, boxes, probs):
        
        res = []
        
        for i in range(len(boxes)):
            
            bbox = list(map(float, boxes[i]))
            bbox.extend([float(probs[i])])
            res.append(bbox)
        
        return res
    
    def predict(self, X, feature_names, meta=None):

        #im_bytes = base64.b64decode(X[0])
        im_bytes = base64.b64decode(X[0,0])
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        model = MTCNN_Model(self.general_parameters, self.model_parameters, self.inference_parameters)
        boxes, probs = model.predict(img, "Deployment")
        result = self.format_result(boxes, probs)
        
        return result

## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "corinthians",
            "ftype": "categorical",
            "values": ["..."]
        }
    ],
    "targets": []
}

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")